### <span style="color: red"> Task 1:</span> Some errors are presented in the code below. Can you identify and corret them?

In [ ]:
import os
import csv

current_dir = os.getcwd()
filename = current_dir+'E1_T1.csv'
with open(filename, 'r', newline=') as f:
    csv_file = csv.reader(f)
    data = list(csv_file)
sentence=''
for word in data:
    sentence = sentence + ' '+ word
print(sentence)
print(type(sentence))

### <span style="color: red"> Task 2:</span> Who has the most runner-up finishes in the histroy of NHL? Create a table with the teams and their runner-up finishes. The table must contain the the following columns:

-Team

-Total number of runner-up finishes

-Years [list of the years they finished as a runner-up]

You have to use Jupyter for the practice and produce a notebook with good readability.

### <span style="color: blue"> Solution:</span>

### <span style="color: red"> Task 3:</span> With the help of matminer, 1. Retrive the 'steel_strength' dataset into a dataframe and clean your dataset in needed; 2. Featurize the compositions of the materials; and 3. Perform correlation analysis and drop feature(s) with an efficient greater than abs(0.8) from your dataframe.

### <span style="color: blue"> Solution:</span>

In [71]:
import matminer
from matminer.datasets import load_dataset
#matminer.datasets.get_available_datasets() #you can check what datasets are available with this line


### <span style="color: red"> Task 4:</span> Setup a private Github repository for this course. Name it MSE1065_\<YourName>_\<YourStudentID>. You will be using this repo for homework submission and course project version control. A good place to get started is [here](https://docs.github.com/en/get-started/quickstart/hello-world). You can choose to use the GitHub via the terminal (recommanded), the [GitHub desktop app](https://desktop.github.com/), or the online GitHub platform. After setting up your repo, make sure to share your repo with [Jae](https://github.com/jae3goals) and [Xiao](github.com/xshang93) by adding us as collobrators.